In [1]:
import datetime
import json
import logging
import os

from xcube.core.store import new_data_store
import numpy as np
import xarray as xr
from zappend.api import zappend


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

LOG = logging.getLogger("MyLogger")

In [2]:
with open("cds-credentials.json") as f:
    credentials = json.load(f)

cds_store = new_data_store(
    "cds",
    **credentials,
    normalize_names=True,
)

In [3]:
store_team = new_data_store(
    "s3",
    root=os.environ["S3_USER_STORAGE_BUCKET"],
    max_depth=4,
    storage_options=dict(
        anon=False,
        key=os.environ["S3_USER_STORAGE_KEY"],
        secret=os.environ["S3_USER_STORAGE_SECRET"],
    ),
)

In [4]:
def get_time_ranges():
    years = np.arange(1981, 2025)
    time_ranges = []
    for year in years:
        for month in range(1, 13):
            time_ranges.append([f"{year}-{month:02}-01", f"{year}-{month:02}-28"])
    return time_ranges

time_ranges = get_time_ranges()
time_ranges

[['1981-01-01', '1981-01-28'],
 ['1981-02-01', '1981-02-28'],
 ['1981-03-01', '1981-03-28'],
 ['1981-04-01', '1981-04-28'],
 ['1981-05-01', '1981-05-28'],
 ['1981-06-01', '1981-06-28'],
 ['1981-07-01', '1981-07-28'],
 ['1981-08-01', '1981-08-28'],
 ['1981-09-01', '1981-09-28'],
 ['1981-10-01', '1981-10-28'],
 ['1981-11-01', '1981-11-28'],
 ['1981-12-01', '1981-12-28'],
 ['1982-01-01', '1982-01-28'],
 ['1982-02-01', '1982-02-28'],
 ['1982-03-01', '1982-03-28'],
 ['1982-04-01', '1982-04-28'],
 ['1982-05-01', '1982-05-28'],
 ['1982-06-01', '1982-06-28'],
 ['1982-07-01', '1982-07-28'],
 ['1982-08-01', '1982-08-28'],
 ['1982-09-01', '1982-09-28'],
 ['1982-10-01', '1982-10-28'],
 ['1982-11-01', '1982-11-28'],
 ['1982-12-01', '1982-12-28'],
 ['1983-01-01', '1983-01-28'],
 ['1983-02-01', '1983-02-28'],
 ['1983-03-01', '1983-03-28'],
 ['1983-04-01', '1983-04-28'],
 ['1983-05-01', '1983-05-28'],
 ['1983-06-01', '1983-06-28'],
 ['1983-07-01', '1983-07-28'],
 ['1983-08-01', '1983-08-28'],
 ['1983-

In [5]:
# extract ERA5 from CDS
variable_names = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "skin_temperature",
    "soil_temperature_level_1",
    "soil_temperature_level_2",
    "soil_temperature_level_3",
    "soil_temperature_level_4",
    "skin_reservoir_content",
    "volumetric_soil_water_layer_1",
    "volumetric_soil_water_layer_2",
    "volumetric_soil_water_layer_3",
    "volumetric_soil_water_layer_4",
    "forecast_albedo",
    "surface_latent_heat_flux",
    "surface_net_solar_radiation",
    "surface_net_thermal_radiation",
    "surface_sensible_heat_flux",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards",
    "potential_evaporation",
    "runoff",
    "total_evaporation",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "surface_pressure",
    "total_precipitation",
    "leaf_area_index_high_vegetation",
    "leaf_area_index_low_vegetation"
]
for time_range in time_ranges:
    file_name = time_range[0].replace("-", "_")
    data_id = f"cubes/aux/era5_land_monthly_means/{file_name}.zarr"
    if store_team.has_data(data_id):
        print(f"ERS5 for {time_range} is already written to {data_id}")
        continue
    print(time_range)
    ds = cds_store.open_data(
        "reanalysis-era5-land-monthly-means:monthly_averaged_reanalysis",
        "dataset:netcdf:cds",
        variable_names=variable_names,
        bbox=[-5, 40, 35, 65],
        spatial_res=0.1,
        time_range=time_range,
    )
    store_team.write_data(ds, data_id, replace=True)
    print(f"ERS5-Land for {time_range} is written to {data_id}")

ERS5 for ['1981-01-01', '1981-01-28'] is already written to cubes/aux/era5_land_monthly_means/1981_01_01.zarr
ERS5 for ['1981-02-01', '1981-02-28'] is already written to cubes/aux/era5_land_monthly_means/1981_02_01.zarr
ERS5 for ['1981-03-01', '1981-03-28'] is already written to cubes/aux/era5_land_monthly_means/1981_03_01.zarr
ERS5 for ['1981-04-01', '1981-04-28'] is already written to cubes/aux/era5_land_monthly_means/1981_04_01.zarr
ERS5 for ['1981-05-01', '1981-05-28'] is already written to cubes/aux/era5_land_monthly_means/1981_05_01.zarr
ERS5 for ['1981-06-01', '1981-06-28'] is already written to cubes/aux/era5_land_monthly_means/1981_06_01.zarr
ERS5 for ['1981-07-01', '1981-07-28'] is already written to cubes/aux/era5_land_monthly_means/1981_07_01.zarr
ERS5 for ['1981-08-01', '1981-08-28'] is already written to cubes/aux/era5_land_monthly_means/1981_08_01.zarr
ERS5 for ['1981-09-01', '1981-09-28'] is already written to cubes/aux/era5_land_monthly_means/1981_09_01.zarr
ERS5 for [

In [6]:
data_ids = store_team.list_data_ids()
data_ids = [data_id for data_id in data_ids if "cubes/aux/era5_land_monthly_means/" in data_id]
data_ids

['cubes/aux/era5_land_monthly_means/1981_01_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_02_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_03_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_04_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_05_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_06_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_07_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_08_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_09_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_10_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_11_01.zarr',
 'cubes/aux/era5_land_monthly_means/1981_12_01.zarr',
 'cubes/aux/era5_land_monthly_means/1982_01_01.zarr',
 'cubes/aux/era5_land_monthly_means/1982_02_01.zarr',
 'cubes/aux/era5_land_monthly_means/1982_03_01.zarr',
 'cubes/aux/era5_land_monthly_means/1982_04_01.zarr',
 'cubes/aux/era5_land_monthly_means/1982_05_01.zarr',
 'cubes/aux/era5_land_monthly_means/1982_06_01.zarr',
 'cubes/aux/era5_land_monthl

In [9]:
# open dataset and check for nan values
for i, data_id in enumerate(data_ids):
    if i % 12 == 0:
        LOG.info(f"Checking {data_id}.")
    ds = store_team.open_data(data_id)
    for var in ds.data_vars:
        vals = ds[var].values
        nans = vals[np.isnan(vals)] 
        perc = nans.size / vals.size * 100 
        # 31.503% is water in the selected bbox -> no data in ERA5-land
        if perc  >  31.504:
            LOG.info(
                f"{data_id} has {perc}% nan"
                f"values for variable name {var}."
            )

2025-06-05 22:09:43,709 - INFO - Checking cubes/aux/era5_land_monthly_means/1981_01_01.zarr.
2025-06-05 22:10:12,820 - INFO - Checking cubes/aux/era5_land_monthly_means/1982_01_01.zarr.
2025-06-05 22:10:42,748 - INFO - Checking cubes/aux/era5_land_monthly_means/1983_01_01.zarr.
2025-06-05 22:11:13,104 - INFO - Checking cubes/aux/era5_land_monthly_means/1984_01_01.zarr.
2025-06-05 22:11:43,075 - INFO - Checking cubes/aux/era5_land_monthly_means/1985_01_01.zarr.
2025-06-05 22:12:13,558 - INFO - Checking cubes/aux/era5_land_monthly_means/1986_01_01.zarr.
2025-06-05 22:12:44,456 - INFO - Checking cubes/aux/era5_land_monthly_means/1987_01_01.zarr.
2025-06-05 22:13:15,515 - INFO - Checking cubes/aux/era5_land_monthly_means/1988_01_01.zarr.
2025-06-05 22:13:47,121 - INFO - Checking cubes/aux/era5_land_monthly_means/1989_01_01.zarr.
2025-06-05 22:14:18,471 - INFO - Checking cubes/aux/era5_land_monthly_means/1990_01_01.zarr.
2025-06-05 22:14:50,358 - INFO - Checking cubes/aux/era5_land_monthly_

In [10]:
final_data_id = "cubes/aux/era5_land_monthly_mean.zarr"
config = {
    "target_dir": f"s3://{os.environ['S3_USER_STORAGE_BUCKET']}/{final_data_id}",
    "target_storage_options": {
        "key": os.environ["S3_USER_STORAGE_KEY"],
        "secret": os.environ["S3_USER_STORAGE_SECRET"],
    },
    "slice_storage_options": {
        "key": os.environ["S3_USER_STORAGE_KEY"],
        "secret": os.environ["S3_USER_STORAGE_SECRET"],              
    }, 
    "force_new": True,
    "logging": True,
    "excluded_variables": ["expver", "number"]
}
zappend(
    (f"s3://{os.environ['S3_USER_STORAGE_BUCKET']}/{data_id}" for data_id in data_ids),
    config=config
)

2025-06-05 22:32:34,750 WARNING Setting 'force_new' is enabled. This will permanently delete existing targets (no rollback).
2025-06-05 22:32:34,750 - WARNING - Setting 'force_new' is enabled. This will permanently delete existing targets (no rollback).
2025-06-05 22:32:35,353 INFO Opening slice dataset from s3://hub-deepesdl37/cubes/aux/era5_land_monthly_means/1981_01_01.zarr
2025-06-05 22:32:35,353 - INFO - Opening slice dataset from s3://hub-deepesdl37/cubes/aux/era5_land_monthly_means/1981_01_01.zarr
2025-06-05 22:32:35,624 INFO Creating target dataset s3://hub-deepesdl37/cubes/aux/era5_land_monthly_mean.zarr
2025-06-05 22:32:35,624 - INFO - Creating target dataset s3://hub-deepesdl37/cubes/aux/era5_land_monthly_mean.zarr
/home/conda/deepesdl37/4f507311-1749109071-36-cubegen/lib/python3.13/site-packages/zappend/processor.py:151: FutureWarning: zarr_version is deprecated, use zarr_format
  target_ds.to_zarr(
2025-06-05 22:32:45,536 INFO Transaction completed.
2025-06-05 22:32:45,536

528

In [4]:
final_data_id = "cubes/aux/era5_land_monthly_mean.zarr"
ds = store_team.open_data(final_data_id)
ds

<xarray.Dataset> Size: 6GB
Dimensions:  (time: 528, lat: 250, lon: 400)
Coordinates:
  * lat      (lat) float64 2kB 64.95 64.85 64.75 64.65 ... 40.25 40.15 40.05
  * lon      (lon) float64 3kB -4.95 -4.85 -4.75 -4.65 ... 34.75 34.85 34.95
  * time     (time) datetime64[ns] 4kB 1981-01-01 1981-02-01 ... 2024-12-01
Data variables: (12/28)
    d2m      (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    e        (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    fal      (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    lai_hv   (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    lai_lv   (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    pev      (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    ...       ...
    swvl3    (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    swvl4    (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    t2m      (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    tp       (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    u10      (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
    v10      (time, lat, lon) float32 211MB dask.array<chunksize=(1, 250, 400), meta=np.ndarray>
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    history:                 2025-06-05T12:32 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [ ]:
# open dataset and check for nan values
final_data_id = "cubes/aux/era5_land_monthly_mean.zarr"
ds = store_team.open_data(final_data_id)
for var in list(ds.keys()):
    LOG.info(f"Checking variable {var}.")
    vals = ds[var].values
    nans = vals[np.isnan(vals)] 
    perc = nans.size / vals.size * 100 
    # 31.503% is water in the selected bbox -> no data in ERA5-land
    if perc  >  31.504:
        LOG.info(
            f"{final_data_id} has {perc}% nan"
            f"values for variable name {var}"
            f"for index {i} to {i+24}."
        )

2025-06-06 06:57:55,016 - INFO - Checking variable d2m.
2025-06-06 06:58:00,614 - INFO - Checking variable e.
2025-06-06 06:58:06,241 - INFO - Checking variable fal.
2025-06-06 06:58:11,880 - INFO - Checking variable lai_hv.
2025-06-06 06:58:17,546 - INFO - Checking variable lai_lv.
2025-06-06 06:58:23,083 - INFO - Checking variable pev.
2025-06-06 06:58:28,816 - INFO - Checking variable ro.
2025-06-06 06:58:34,348 - INFO - Checking variable skt.
2025-06-06 06:58:39,848 - INFO - Checking variable slhf.
2025-06-06 06:58:45,317 - INFO - Checking variable sp.
2025-06-06 06:58:50,871 - INFO - Checking variable src.
2025-06-06 06:58:56,386 - INFO - Checking variable sshf.
2025-06-06 06:59:01,946 - INFO - Checking variable ssr.
2025-06-06 06:59:07,418 - INFO - Checking variable ssrd.
2025-06-06 06:59:12,941 - INFO - Checking variable stl1.
2025-06-06 06:59:18,409 - INFO - Checking variable stl2.
2025-06-06 06:59:24,028 - INFO - Checking variable stl3.
2025-06-06 06:59:29,544 - INFO - Checkin